# Introduction to Spark

## Big data

Big data is a general term referring to several challenges encountered when dealing with a large amount of data. These challenges are known as **the 5 V's** - Volume, Variety, Velocity, Variability and Veracity. Traditionally, the 5 V's were solved by the **scale-up** approach, which means upgrading the resources - better servers, faster connections, larger memories, etc. This is now replaced by the **scale-out** approach, which is based on **parallelizing** tasks over a **cluster** of "weak" servers.

It is important to notice that the 5 V's are **infrastructural** challenges and are not necessarily related to any business or analytical problem. This is important to notice, because the confusion between the two is very common, making people think that data science and big data are similar expertise. Our course is not about big data, however several concepts are of special importance in this subject, so I give below brief explanations and references.

* [**Cluster**][cluster] - a set of loosely or tightly connected computers that work together so that, in many respects, they can be viewed as a single system. Any cluster has a **master** (or main) and **nodes** (or slaves), which offer two main advantages - **Fault-tolerance** and **Data locality**. 
* [**MapReduce**][mapreduce] - a programming model and an associated implementation for processing and generating big data sets with a parallel, distributed algorithm on a cluster.
* [**Hadoop**][hadoop] - an open-source software framework used for distributed storage and processing of big data sets using the MapReduce programming model.
    * **Hadoop Data File System (HDFS)** - a distributed, scalable, and portable file system for the Hadoop framework.
    * **Hadoop ecosystem** - collection of additional software packages that can be installed on top of or alongside Hadoop (e.g. Yarn, Pig, Hive, Sqoop, Mahout, etc.). Many of the big data tools aim to **abstract** the parallelization.
    * **Hadoop distribution** - companies providing Hadoop-based software, support, services and training (e.g. Cloudera, Hortonworks, etc.)

[cluster]: https://en.wikipedia.org/wiki/Computer_cluster "Computer cluster - Wikipedia"
[mapreduce]: https://en.wikipedia.org/wiki/MapReduce "MapReduce - Wikipedia"
[hadoop]: https://en.wikipedia.org/wiki/Apache_Hadoop "Hadoop - Wikipedia"
[hdfs]: https://en.wikipedia.org/wiki/Apache_Hadoop#HDFS "HDFS - Wikipedia"

## Spark

[Spark][spark], like Hadoop itself, is a framework for programming with an abstraction of the map-reduce paradigm. Its main data structure (RDD) allows better utilization of the memory of the nodes, and this made it very popular in recent years. Spark was originally part of the Hadoop ecosystem, however it was so useful, that eventually it was decided to make it available as a stand-alone framework. Spark is written in [Scala][scala], but it suports APIs for Java, R and of course Python.

Spark is made of 5 building blocks:

* Spark core - the fundamentals components of the language. It provides distributed task dispatching, scheduling, and basic I/O functionalities, exposed through an API centered on the RDD abstraction.
* Spark SQL - tools for working with DataFrames. It provides an API for embedding SQL scripts, as well as connections with an ODBC/JDBC server.
* Spark streaming - facilitates tasks witha a data stream. It ingests data in mini-batches and performs RDD transformations on those mini-batches of data.
* Spark MLlib - distributed versions of various machine learning (ML) algorithms.
* Spark GraphX - graph processing framework.

In our course, we will explore 3 of the 5 - Spark core, Spark SQL and Spark MLlib, and we will do it using the Python API - **pyspark**.

[spark]: https://en.wikipedia.org/wiki/Apache_Spark "Apache Spark - Wikipedia"
[scala]: https://en.wikipedia.org/wiki/Scala_(programming_language) "Scala - Wikipedia"

### Spark core

#### RDD

**Resilient Distributed Dataset (RDD)** is the main data object in Spark and it is an abstraction of the data parallelization. This means that we can work with a single RDD, where in fact its data, as well as its processing, may be distributed in the cluster.

Data sharing is slow in MapReduce due to replication, serialization, and disk IO (Actually, most Hadoop applications spend more than 90% of the time doing HDFS read-write operations.). Recognizing this problem, RDDs support **in-memory** processing computation. This means, it stores the state of memory as an object across the jobs and the object is sharable between those jobs.

Two technical comments:

* RDDs are immutable, which has a great influence on the appearence of Spark code.
* If the elements of an RDD are tuples (which is a Spark data type, equivalent to Python tuples of length 2), then each tuple is automatically recognized as a pair of a **key** and a **value**.

#### Transformations and actions

RDD **transformations** are operations applied on RDDs to yield a new RDD. On the other hand, **actions** are operations applied on RDDs to yield a non-RDD result (number, string, list, etc.). 

Here are some examples:

* Transformations:
    * _map(func)_ - Returns a new distributed dataset, formed by passing each element of the source through a function func.
    * _flatMap(func)_ - Similar to map, but each input item can be mapped to 0 or more output items (so func should return a Seq rather than a single item).
    * _filter(func)_ - Returns a new dataset formed by selecting those elements of the source on which func returns true.
    * _union(otherDataset)_ - Returns a new dataset that contains the union of the elements in the source dataset and the argument.
    * _groupByKey()_ - When called on a dataset of (K, V) pairs, returns a dataset of (K, Iterable(V)) pairs.
    * _reduceByKey(func)_ - When called on a dataset of (K, V) pairs, returns a dataset of (K, V) pairs where the values for each key are aggregated using the given reduce function func, which must be of type (V, V) ⇒ V.
    * _sortByKey([ascending])_ - When called on a dataset of (K, V) pairs where K implements Ordered, returns a dataset of (K, V) pairs sorted by keys in ascending or descending order, as specified in the Boolean ascending argument.
* Actions:
    * _reduce(func)_ - Aggregate the elements of the dataset using a function func (which takes two arguments and returns one). The function should be commutative and associative so that it can be computed correctly in parallel.
    * _count()_ - Returns the number of elements in the dataset.
    * _take(n)_ - Returns an array with the first n elements of the dataset. 
    * _saveAsTextFile(path)_ - Writes the elements of the dataset as a text file (or set of text files) in a given directory in the local filesystem, HDFS or any other Hadoop-supported file system. Spark calls _toString()_ on each element to convert it to a line of text in the file.

Two technical comments:

* In most cases one applies a chain of transformations which ends with an action. Each RDD in such dependency chain has a pointer (dependency) to its parent RDD. Spark is **lazy**, so nothing will be executed until an action will trigger the chain. Therefore, RDD transformation is not a set of data but is a step in a program (might be the only step) telling Spark how to get data and what to do with it.
* Spark is written in Scala, which does not support some of the functionalities of Python. This is why the Python API offers some additional transformations which are not part of the core functionalities, but a wrapper of them. 

> **WARNING:** There is an action called _collect()_ which is similar to the _take()_ action, but returns **all** the elements of the RDD. This action collects the relevant elements to the master of the cluster, and can easily crush the system. This is why it is usually useful after a filter or other operation that returns a sufficiently small subset of the data.

### Spark SQL

Spark SQL is a Spark module for structured data processing. 

Unlike the basic Spark RDD API, the interfaces provided by Spark SQL provide Spark with more information about the structure of both the data and the computation being performed. Internally, Spark SQL uses this extra information to perform extra optimizations. There are several ways to interact with Spark SQL including plain SQL and the **Dataframe** API (generalized as **Dataset** in recent versions). When computing a result the same execution engine is used, independent of which API/language you are using to express the computation.

#### Dataframe

Dataframe is a special type of RDD, and as such it supports an additional set of attributes and methods:

* Attributes
    * _column_ - Returns all column names as a list.
    * _rdd_ - Returns the content as an RDD (of _Row_ elements).
    * _schema_ - Returns the schema of this DataFrame (as a _StructType_).
* Methods
    * _crosstab(col1, col2)_ - Computes a pair-wise frequency table of the given columns (pivot table).
    * _drop(\*cols)_ - Returns a new DataFrame that drops the specified column.
    * _head(n=None)_ - Returns the first n rows.
    * _orderBy(\*cols)_ - Returns a new DataFrame sorted by the specified column(s).
    * _printSchema()_ - Prints out the schema in a tree format.
    * _where(condition)_ - an alias for _filter()_

### Spark MLlib

MLlib is Spark’s machine learning (ML) library, and its goal is to make practical machine learning scalable and easy. At a high level, it provides tools for:

* ML Algorithms - common learning algorithms such as classification, regression, clustering, and collaborative filtering
* Featurization - feature extraction, transformation, dimensionality reduction, and selection
* Pipelines - tools for constructing, evaluating, and tuning ML Pipelines
* Utilities - linear algebra, statistics, data handling, etc.

## Working with Spark

### SparkConf, SparkContext & SparkSession

The first thing a Spark program must do is to create a **SparkContext** object (traditionally symbolized as the variable _sc_), which tells Spark how to access a cluster. To create a SparkContext you first need to build a **SparkConf** object that contains information about your application. A SparkContext is what you need for working in the Spark core elements, however the entry point into Spark SQL functionality is the **SparkSession** object (traditionally symbolized as the variable _spark_).

For illustration, if you have Spark installed on your local machine, the following script will have to be inserted at the beginning of your PySpark code.

In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

conf = SparkConf().setAppName('appName').setMaster('local')
sc = SparkContext(conf=conf)
spark = SparkSession(sc)

### Databricks

In order to work with Spark one has to install it and configure a cluster. This is not a simple task, so Spark's creators founded a second company called [databricks][databricks], which offers a cloud-based service for working with Spark. This is a great place to start working with Spark, as it bypasses many of the getting-started obstacles. For example, the objects _sc_ and _spark_ are available by default, and useful development packages like NumPy and pandas are also available.

[databricks]: https://databricks.com/ "databricks homepage"

> **NOTE:** Part of the class materials and all of the exercises will be done in the databricks platform, so you must create your own account. Please sign up [here][signup] (the community edition is free).

[signup]: https://databricks.com/try-databricks "databricks signup"

#### DBFS - Databricks File System

Databricks is a cloud-based service, and it requires that your data will be uploaded to the cloud as well. DBFS is the distributed file system that comes installed on Spark Clusters in Databricks. This is not a very intuitive system, butu an explanation of the uploading and loading processes are documented [here][dbfs].

[dbfs]: https://docs.databricks.com/user-guide/importing-data.html "importing data"